In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import string
from sklearn.model_selection import train_test_split


In [3]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en
!{sys.executable} pip install more-itertools

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-12-22 09:07:07.911809: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-12-22 09:07:07.911847: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
C:\Python\python.exe: can't open file 'E:\DM_HW2Kaggle\pip': [Errno 2] No such file or directory


In [31]:
!pip install more-itertools

In [32]:
import itertools

In [5]:
from nltk.corpus import wordnet

In [6]:
#import data
df_iden = pd.read_csv("data_identification.csv")
df_iden


,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train
...,...,...
1867530,0x227e25,train
1867531,0x293813,train
1867532,0x1e1a7e,train
1867533,0x2156a5,train


In [7]:
#tweet-id - test
df_test= df_iden[df_iden['identification'] == 'test']
df_test=df_test.dropna()
df_test

,tweet_id,identification
0,0x28cc61,test
3,0x2db41f,test
15,0x2466f6,test
23,0x23f9e9,test
31,0x1fb4e1,test
...,...,...
1867495,0x2c4dc2,test
1867496,0x31be7c,test
1867500,0x1ca58e,test
1867515,0x35c8ba,test


In [8]:
#tweet-id - train
df_train = df_iden[df_iden['identification'] == 'train']
df_train

,tweet_id,identification
1,0x29e452,train
2,0x2b3819,train
4,0x2a2acc,train
5,0x2a8830,train
6,0x20b21d,train
...,...,...
1867530,0x227e25,train
1867531,0x293813,train
1867532,0x1e1a7e,train
1867533,0x2156a5,train


In [10]:
#import -emotion
emo_df = pd.read_csv("emotion.csv")

In [11]:
emo_df

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation
...,...,...
1455558,0x38dba0,joy
1455559,0x300ea2,joy
1455560,0x360b99,fear
1455561,0x22eecf,joy


In [2]:
#import the json file
tweets_df = pd.read_json(r'tweets_DM.json',lines=True)

NameError: name 'pd' is not defined

In [13]:
tweets_df

,_score,_index,_source,_crawldate,_type
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets
...,...,...,...,...,...
1867530,827,hashtag_tweets,"{'tweet': {'hashtags': ['mixedfeeling', 'butim...",2015-05-12 12:51:52,tweets
1867531,368,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x29d0...",2017-10-02 17:54:04,tweets
1867532,498,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2a6a...",2016-10-10 11:04:32,tweets
1867533,840,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x24fa...",2016-09-02 14:25:06,tweets


In [20]:
# convert column _source to be list obj.
data = tweets_df['_source'].tolist() # extract text coulum to be list obj
df = pd.DataFrame.from_dict(pd.json_normalize(data), orient='columns') # normalize data
tweets_DM= pd.concat([tweets_df, df], axis=1) # concat them again

In [28]:
type(tweets_DM)

pandas.core.frame.DataFrame

In [43]:
# use to clean tweet.text

def cleaning_text(text):
    
    text = text.strip() # remove white space
    text = re.sub(r'http\S+', "", text)    # remove urls
    text = re.sub(r'@\w+','',text)         # remove mentions
    text = re.sub(r'#', '', text)       # remove hastags \w+
    text = re.sub(r'<\w+','', text)       # remove html tags
    text = re.sub(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', '', text) 
    text = re.sub(r'>', '', text)
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text)) # remove multiple letters
    text = text.lower()  #convert to lower case
    text = re.sub('[!,*)@#%(&$_?.^+->:;/\"]', '', text)
    text = re.sub(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', text)
                  
    return text

In [44]:
tweets_DM['clean_text']=tweets_DM['tweet.text'].apply(lambda x: cleaning_text(x))

In [1]:
tweets_DM

NameError: name 'tweets_DM' is not defined

In [47]:
# creat a training set 
# merge with df_emo
tweets_DM_train = tweets_DM.merge(emo_df, how='inner',left_on = 'tweet.tweet_id', right_on = 'tweet_id')

In [48]:
tweets_DM_train

,_score,_index,_source,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text,clean_text,tweet_id,emotion
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",people who post add me on snapchat must be deh...,0x376b20,anticipation
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",as we see trump is dangerous to freepress aro...,0x2d5350,sadness
2,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,now issa is stalking tasha 😂😂,0x1cd5b0,fear
3,120,hashtag_tweets,"{'tweet': {'hashtags': ['authentic', 'LaughOut...",2015-06-11 04:44:05,tweets,"[authentic, LaughOutLoud]",0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,thx for the best time tonight what stories h...,0x1d755c,joy
4,1021,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2c91...",2015-08-18 02:30:07,tweets,[],0x2c91a8,Still waiting on those supplies Liscus. <LH>,still waiting on those supplies liscus,0x2c91a8,anticipation
...,...,...,...,...,...,...,...,...,...,...,...
1455558,94,hashtag_tweets,"{'tweet': {'hashtags': ['NoWonder', 'Happy'], ...",2016-12-26 02:44:07,tweets,"[NoWonder, Happy]",0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,i'm so happy nowonder the name of this show ha...,0x321566,joy
1455559,627,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x3895...",2015-04-01 08:14:56,tweets,[],0x38959e,In every circumtance I'd like to be thankful t...,in every circumtance i'd like to be thankful t...,0x38959e,joy
1455560,274,hashtag_tweets,"{'tweet': {'hashtags': ['blessyou'], 'tweet_id...",2016-11-17 23:46:22,tweets,[blessyou],0x2cbca6,there's currently two girls walking around the...,there's currently two girls walking around the...,0x2cbca6,joy
1455561,840,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x24fa...",2016-09-02 14:25:06,tweets,[],0x24faed,"Ah, corporate life, where you can date <LH> us...",ah corporate life where you can date using ju...,0x24faed,joy


In [50]:
df_test=tweets_DM.merge(df_test, how='inner',left_on = 'tweet.tweet_id', right_on = 'tweet_id')

In [51]:
df_test

,_score,_index,_source,_crawldate,_type,tweet.hashtags,tweet.tweet_id,tweet.text,clean_text,tweet_id,identification
0,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",confident of your obedience i write to you kno...,0x28b412,test
1,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets,[],0x2de201,"""Trust is not the same as faith. A friend is s...",trust is not the same as faith a friend is som...,0x2de201,test
2,66,hashtag_tweets,"{'tweet': {'hashtags': ['materialism', 'money'...",2015-09-09 09:22:55,tweets,"[materialism, money, possessions]",0x218443,When do you have enough ? When are you satisfi...,when do you have enough when are you satisfie...,0x218443,test
3,104,hashtag_tweets,"{'tweet': {'hashtags': ['GodsPlan', 'GodsWork'...",2015-10-10 14:33:26,tweets,"[GodsPlan, GodsWork]",0x2939d5,"God woke you up, now chase the day #GodsPlan #...",god woke you up now chase the day godsplan god...,0x2939d5,test
4,310,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2628...",2016-10-23 08:49:50,tweets,[],0x26289a,"In these tough times, who do YOU turn to as yo...",in these tough times who do you turn to as you...,0x26289a,test
...,...,...,...,...,...,...,...,...,...,...,...
411967,602,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2913...",2016-12-10 18:01:00,tweets,[],0x2913b4,"""For this is the message that ye heard from th...",for this is the message that ye heard from the...,0x2913b4,test
411968,598,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2a98...",2015-01-04 14:40:55,tweets,[],0x2a980e,"""There is a lad here, which hath five barley l...",there is a lad here which hath five barley loa...,0x2a980e,test
411969,827,hashtag_tweets,"{'tweet': {'hashtags': ['mixedfeeling', 'butim...",2015-05-12 12:51:52,tweets,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,when you buy the last tickets remaining for a...,0x316b80,test
411970,368,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x29d0...",2017-10-02 17:54:04,tweets,[],0x29d0cb,I swear all this hard work gone pay off one da...,i swear all this hard work gone pay off one da...,0x29d0cb,test


In [52]:
y_train = tweets_DM_train['emotion']

In [53]:
y_train

0          anticipation
1               sadness
2                  fear
3                   joy
4          anticipation
               ...     
1455558             joy
1455559             joy
1455560             joy
1455561             joy
1455562             joy
Name: emotion, Length: 1455563, dtype: object

In [55]:
# train dataset -tokenizing
#tokenizer - - using bag of words BOW 
from sklearn.feature_extraction.text import CountVectorizer

# build analyzers (bag-of-words)
BOW= CountVectorizer(tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW.fit(tweets_DM_train['clean_text'])

train_data_BOW_train = BOW.transform(tweets_DM_train['clean_text'])
train_data_BOW_test = BOW.transform(df_test['clean_text'])

C:\Python\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Python\lib\site-packages\sklearn\feature_extraction\text.py:1208: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [56]:
#apply naive bayes model
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
naivebay = MultinomialNB()
naive=naivebay.fit(train_data_BOW_train, y_train)

In [57]:
result1 = naive.predict(train_data_BOW_test)

In [59]:
result1.shape

(411972,)

In [60]:
df_test['emotion'] = result1

In [62]:
answer=df_test[['tweet_id','emotion']]

In [63]:
answer

,tweet_id,emotion
0,0x28b412,anticipation
1,0x2de201,anticipation
2,0x218443,joy
3,0x2939d5,anticipation
4,0x26289a,anticipation
...,...,...
411967,0x2913b4,anticipation
411968,0x2a980e,anticipation
411969,0x316b80,joy
411970,0x29d0cb,joy


In [1]:
#export the file
answer.to_csv('fourteenth')

NameError: name 'answer' is not defined